<h2>Regression Models with Keras. Part C</h2>
<h3>Objective for this Notebook:</h3>    
Repeat Regression Models with Keras Part B but use 100 epochs this time for training.   

How does the mean of the mean squared errors compare to that from Step B?

<h3>Concrete Data:</h3>    
The data can be found here: 
https://cocl.us/concrete_data
   
 
       


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Prepare the data: Download, Clean and Split the Dataset     
2. <a href="#item32">Import Keras</a>  
3. <a href="#item33">Build a Neural Network</a>  
4. <a href="#item34">Train and Test the Network</a>  
5. <a href="#item35">Evaluate the model</a>      

</font>
</div>


<a id="item31"></a>

## 1. Prepare the data: Download, Check and Split the Dataset

#### 1.1 Download the data 

Import the <em>pandas</em> and the Numpy libraries.


In [1]:
# Uncomment the following if running on desktop:
#!pip install numpy==1.21.4
#!pip install pandas==1.3.4
#!pip install keras==2.1.6

In [2]:
import pandas as pd
import numpy as np

Download the data and read it into a <em>pandas</em> dataframe.

In [3]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:

1. Cement
2. Blast Furnace Slag
3. Fly Ash
4. Water
5. Superplasticizer
6. Coarse Aggregate
7. Fine Aggregate


#### 1.2 Check the data 

Check how many data points we have.

In [4]:
concrete_data.shape

(1030, 9)

Check the dataset for any missing values.

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

#### 1.3 Split the data into predictors and target


In [7]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Sanity check of the predictors and the target dataframes.


In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Normalize the data by substracting the mean and dividing by the standard deviation.


In [10]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Save the number of predictors to *n_cols* since we will need this number when building our network.


In [11]:
n_cols = predictors_norm.shape[1] # number of predictors

<a id="item32"></a>

## 2. Import Keras 
Import Keras and the packages from the Keras library that we will need to build our regression model.

In [12]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [13]:
from keras.models import Sequential
from keras.layers import Dense

<a id="item33"></a>

## 3. Build a Neural Network

Define a function that defines our regression model for us so that we can conveniently call it to create our model.

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error as the loss function.


In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Call the function to create our model.


In [15]:
# build the model
model = regression_model()

<a id="item34"></a>

## 4. Train and Test the Network


4.1. Randomly split the data into a training and test sets by holding 30% of the data for testing.  
4.2. Train the model on the training data using 100 epochs.

In [16]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)



Train on 721 samples, validate on 309 samples
Epoch 1/100


2024-01-06 16:24:13.707979: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2024-01-06 16:24:13.712747: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394295000 Hz
2024-01-06 16:24:13.713555: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555e70be15b0 executing computations on platform Host. Devices:
2024-01-06 16:24:13.713611: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2024-01-06 16:24:13.786856: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

 - 0s - loss: 1679.3390 - val_loss: 1207.3205
Epoch 2/100
 - 0s - loss: 1660.0087 - val_loss: 1197.5431
Epoch 3/100
 - 0s - loss: 1640.0461 - val_loss: 1187.4241
Epoch 4/100
 - 0s - loss: 1619.4950 - val_loss: 1176.7830
Epoch 5/100
 - 0s - loss: 1598.0039 - val_loss: 1165.6024
Epoch 6/100
 - 0s - loss: 1575.1224 - val_loss: 1153.7292
Epoch 7/100
 - 0s - loss: 1551.4470 - val_loss: 1141.0121
Epoch 8/100
 - 0s - loss: 1525.8586 - val_loss: 1127.3273
Epoch 9/100
 - 0s - loss: 1498.8880 - val_loss: 1112.7618
Epoch 10/100
 - 0s - loss: 1469.7266 - val_loss: 1097.3077
Epoch 11/100
 - 0s - loss: 1438.9564 - val_loss: 1080.7729
Epoch 12/100
 - 0s - loss: 1406.5471 - val_loss: 1063.0793
Epoch 13/100
 - 0s - loss: 1372.0701 - val_loss: 1044.1483
Epoch 14/100
 - 0s - loss: 1336.2350 - val_loss: 1024.1089
Epoch 15/100
 - 0s - loss: 1297.8051 - val_loss: 1002.9906
Epoch 16/100
 - 0s - loss: 1258.4231 - val_loss: 980.6749
Epoch 17/100
 - 0s - loss: 1217.6063 - val_loss: 957.5885
Epoch 18/100
 - 0s -

<a id="item35"></a>

## 5. Evaluate the model

5.1. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength.    

In [17]:
## evaluate the model
from sklearn.metrics import mean_squared_error

# y_pred are the predictions of the model with the test data 
y_pred = model.predict(predictors_norm)

# Calculate Mean Squared Error (MSE) 
mse = mean_squared_error(target, y_pred)

print(f'Mean Squared Error: {mse}')

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


Mean Squared Error: 175.50722778361376


5.2. Repeat steps 4.1, 4.2 and 5.1, 50 times, i.e., create a list of 50 mean squared errors.

In [18]:
num_repeats = 50
mse_list = []
for _ in range(num_repeats):
    
    model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=0)
    y_pred = model.predict(predictors_norm)
    mse = mean_squared_error(target, y_pred)
    # Add MSE to the list
    mse_list.append(mse)

print("List of Mean Squared Errors:")
print(mse_list)

List of Mean Squared Errors:
[99.79513585368106, 80.94396168684901, 70.99973009730063, 65.83502999432223, 62.59416328585424, 51.02493607982759, 47.1399277177244, 45.709023050745515, 45.486046610221024, 45.46562378095759, 45.03510384595953, 45.332249045187204, 45.20503143119364, 45.81519820084369, 45.666763524766225, 46.21489246316569, 46.692144329833425, 46.60699408121517, 46.945341248050795, 47.48152453014371, 48.13290954828333, 48.35653661056986, 48.47724313663563, 47.84034800254249, 47.177404933632566, 47.014786131880335, 46.87983551608556, 46.289899999439434, 46.5321495124165, 46.315854782161495, 45.62894726997997, 45.57316551580885, 44.842546127878684, 44.65086970458784, 44.76527432246633, 44.65206058911554, 44.549364048224554, 44.68602243590618, 45.01094630109812, 44.87680185716431, 44.35436687885149, 43.6476247347145, 42.94696185912526, 43.19993841767822, 43.22313651884332, 43.38344416037736, 43.72607455634958, 43.635586458100015, 44.3426255975094, 44.384818536343616]


5.3. Report the mean and the standard deviation of the mean squared errors.

In [19]:
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

print(f"Mean of Mean Squared Errors: {mean_mse}")
print(f"Standard Deviation of Mean Squared Errors: {std_mse}")

Mean of Mean Squared Errors: 48.70172729843226
Standard Deviation of Mean Squared Errors: 10.16797378110146


Mean of Mean Squared Errors: 48.70172729843226  
Standard Deviation of Mean Squared Errors: 10.16797378110146

Increasing the number of epochs to 100 results in a significant decrease in the mean MSE (from 68.37 to 48.70). The reduction in standard deviation indicates greater consistency in performance across different runs.